# Finetune a T5 Transformer model on the California BillSum dataset

In [ ]:
!pip install -q transformers datasets evaluate rouge_score accelerate

# Load BillSum Dataset from the Datasets library

In [ ]:
from datasets import load_dataset

billsum = load_dataset("billsum", split = "ca_test")

In [ ]:
# Split the dataset into a train and test set
billsum = billsum.train_test_split(test_size=0.2)

In [ ]:
# Look at the data
print(billsum)

# The text column is our model input
# The summary column is our model target
print("ORIGINAL TEXT:\n", billsum['train']['text'][0], "\n")
print("SUMMARY:\n", billsum['train']['summary'][0], "\n")

In [ ]:
# Preprocessing
## Load Model
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
# Check Model
print(model)

In [ ]:
for name, param in model.named_parameters():
    print(f"Parameter name: {name}")
    print(f"Requires gradients: {param.requires_grad}")
    print(f"Parameter shape: {param.shape}")
    print("=" * 30)

In [ ]:
# # Optional: Specify layers to fine tune and which to freeze by setting requires grad to true and false
layers_to_fine_tune = ['classifier.weight','classifier.bias']

# Freeze layers
for name, param in model.named_parameters():
    if not any(layer_name in name for layer_name in layers_to_fine_tune):
        param.requires_grad = False

# Unfreeze Fine-tune layers
for name, param in model.named_parameters():
    if any(layer_name in name for layer_name in layers_to_fine_tune):
        param.requires_grad = True

# Preprocessing

We need to create a preprocess function that we will apply to every instance in the dataset. The preprocess function needs to:

1. Prefix input with a prompt so T5 knows it's performing a summarisation task.

2. Use the keyword `text_target` argument when tokenizing labels

3. Truncate sequences to be no longer than the maximum length set by the `max_length` parameter

After applying the preprocessing to every instance in the dataset, we create batches of examples using a data collator function. This is because model training must be in batches.

In [ ]:
def preprocess_function(examples, prefix: str = "summarize: "):
  # prepend the prefix to every instance in the dataset
  inputs = [prefix + doc for doc in examples["text"]]

  # Tokenize the text input and convert them into tensors
  model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

  # Tokenize the labels and convert them into tensors
  labels = tokenizer(text_target=examples['summary'], max_length=123, truncation=True)

  model_inputs["labels"] = labels["input_ids"]

  return model_inputs

In [ ]:
# Apply preprocessing over entire dataset - batched = True process multiple elements of the datasets
tokenized_billsum = billsum.map(preprocess_function, batched = True,)

In [ ]:
tokenized_billsum

In [ ]:
# Create a batch of examples, with dynamic padding. Use the appropriate collator function
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer, model = model_checkpoint,)

# Evaluate & Prepare DataLoaders
We require monitoring a metric during training to see how well our model is doing. We use the `evaluate` library to load an evaluation metric - we use the ROUGE metric.

With the loss metric defined, we must define a function that takes model predictions and labels and computes the loss metric. This is usually called the `compute_metrics` function.

In [ ]:
tokenized_billsum_trch = tokenized_billsum.remove_columns(["text", "summary", "title"])

tokenized_billsum_trch.set_format("torch")
tokenized_billsum_trch

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_billsum_trch['train'], shuffle = True, batch_size = 16, collate_fn=data_collator
)

test_dataloader = DataLoader(
    tokenized_billsum_trch['test'], shuffle = True, batch_size = 16, collate_fn=data_collator
)

In [ ]:
import pprint
batch = next(iter(train_dataloader))

#print(batch)
print(len(train_dataloader))
print(f"input_ids batch shape: {batch.input_ids.shape}")
print(f"attention_mask batch shape: {batch.attention_mask.shape}")
print(f"labels batch shape: {batch.labels.shape}")

In [ ]:
import evaluate

rouge = evaluate.load("rouge")

# Train Model using PyTorch

In [ ]:
from transformers import AdamW, get_scheduler
import torch
import numpy as np

# Define Optimiser
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)

# Define Loss Function
def compute_metrics(predictions, labels):

  # Convert token ids to tokens
  decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

  # If the label is -100 then it's a pad token and we assign that token id to <PAD>
  labels = torch.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

  # Count non padding tokens
  prediction_lens = [torch.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]

  result['gen_len'] = torch.stack(prediction_lens).float().mean()

  return {k: v for (k,v) in result.items()}

# Initialize variables to track the best model
best_loss = float('inf')
best_checkpoint_path = None

# Collect Statistics
train_loss = []
train_metrics = []
test_metrics = []

## Place training on a GPU
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)
model.to(device)

# Define Learning Rate Scheduler
num_epochs = 4
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [ ]:
# Set up a list to store checkpoints
checkpoint_paths = []

model.train()

# Training loop
for epoch in range(num_epochs):
    for i, batch in enumerate(train_dataloader):
        # Set Gradients to 0
        optimizer.zero_grad()

        # Perform a forward model pass
        ## Put the batch onto a GPU
        batch = {k: v.to(device) for (k, v) in batch.items()}

        ## Forward Pass
        outputs = model(**batch)

        # Compute Loss
        loss = outputs.loss

        # Compute Metric
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        labels = batch["labels"]

        metrics = compute_metrics(predictions=predictions, labels=labels)

        # Store Metrics
        train_loss.append(float(loss))
        train_metrics.append(metrics)

        # Backward pass to update parameters
        ## Compute fradients with respect to model parameters
        loss.backward()

        # Optimizer step
        ## Use the computed gradients to update the model parameters - adjust parameters in the direction that reduces the loss
        optimizer.step()

        # Update Learning Rate - according to a schedule. This adjusts learning rate dynamically
        lr_scheduler.step()

        # Print Progress
        print(f"epoch {epoch} batch_number {i} loss {loss} metrics {metrics}")

    # Save checkpoint at certain intervals
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
        'metrics': metrics,
        # Add other relevant information if needed
    }
    checkpoint_path = f'checkpoint_epoch_{epoch}_batch_{i}.bin'
    torch.save(checkpoint, checkpoint_path)
    checkpoint_paths.append(checkpoint_path)

    # Update best_loss and best_checkpoint_path if needed
    if loss < best_loss:
        best_loss = loss
        best_checkpoint_path = checkpoint_path




In [ ]:
# Load the best model checkpoint
best_checkpoint = torch.load(best_checkpoint_path)

model.load_state_dict(best_checkpoint['model_state_dict'])
optimizer.load_state_dict(best_checkpoint['optimizer_state_dict'])

best_epoch = best_checkpoint['epoch']
best_loss = best_checkpoint['loss']
best_metrics = best_checkpoint['metrics']


In [ ]:
print(best_epoch)
print(best_loss)
print(best_metrics)

In [ ]:
model.eval()

eval_metrics = []
for epoch in range(num_epochs):
  for i, batch in enumerate(test_dataloader):
    # Perform a forward model pass
    ## Put the batch onto a GPU
    batch = {k: v.to(device) for (k, v) in batch.items()}

    ## Forward Pass - Set no grad because we don't want to update parameters in validation
    with torch.no_grad():
        outputs = model(**batch)

    # Compute Metric
    logits = outputs.logits
    predictions = torch.argmax(logits, dim = -1)
    labels = batch["labels"]

    metrics = compute_metrics(predictions=predictions, labels=labels)

    # Store Metrics
    eval_metrics.append(metrics)

    # Print Progress
    print(f"epoch {epoch} batch_number {i} metrics {metrics}")


# Save and Load Tokenizer and Model

In [ ]:
# Suggested from Docs: https://huggingface.co/transformers/v1.2.0/serialization.html
# Save Tokenizer and Model
import os

output_dir = "./billsum_summariser_model/"
os.makedirs(output_dir, exist_ok=True)

# Step 1: Save a model, configuration and vocabulary that you have fine-tuned

# If we have a distributed model, save only the encapsulated model
# (it was wrapped in PyTorch DistributedDataParallel or DataParallel)
model_to_save = model.module if hasattr(model, 'module') else model

# If we save using the predefined names, we can load using `from_pretrained`
WEIGHTS_NAME = "pytorch_model.bin"
CONFIG_NAME = "config.json"
output_model_file = os.path.join(output_dir, WEIGHTS_NAME)
output_config_file = os.path.join(output_dir, CONFIG_NAME)

torch.save(model.state_dict(), output_model_file)
model.config.to_json_file(output_config_file)
tokenizer.save_pretrained(output_dir)

In [ ]:
# Load Model
pretrained_loaded_model = AutoModelForSeq2SeqLM.from_pretrained("billsum_summariser_model")

# Inference

In [ ]:
text = "summarize: The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share. And no one making under $400,000 per year will pay a penny more in taxes."

In [ ]:
# Inference
## Tokenize inputs
inputs = tokenizer(text, return_tensors="pt").input_ids
print(inputs)

In [ ]:
outputs = pretrained_loaded_model.generate(inputs,
                                           max_new_tokens=35,
                                           do_sample=False,
                                           )
print(outputs)

In [ ]:
import pprint
original_text = {"original_text": text}
summary_text = {"summary_text": tokenizer.decode(outputs[0],
                                                 skip_special_tokens=True
                                                 )}

pprint.pprint(original_text)
pprint.pprint(summary_text)